In [1]:
import xmlrpc.client
import pandas as pd

# Mostrar todas las columnas y todas las filas completas en pantalla
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)



# ===============================
# 🔗 Conexión con Odoo 8
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
if not uid:
    raise Exception("❌ No se pudo autenticar. Revisa usuario/contraseña/URL.")

models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# 📋 Obtener campos disponibles (sin 'attributes')
# ===============================
fields = models.execute_kw(
    db, uid, password,
    'res.users', 'fields_get', []
)

# Filtrar solo campos de grupos
group_fields = [f for f in fields if f.startswith('in_group_') or f.startswith('sel_groups_')]

# Campos base para identificar usuarios
base_fields = ['id', 'name', 'login', 'email', 'active']
fields_to_read = base_fields + group_fields

print(f"Total de campos a leer: {len(fields_to_read)}")

# ===============================
# 🔍 Obtener IDs de todos los usuarios
# ===============================
user_ids = models.execute_kw(
    db, uid, password,
    'res.users', 'search',
    [[['active', 'in', [True]]]]
)

# ===============================
# 📦 Leer los datos en lotes
# ===============================
records = []
batch_size = 100
for i in range(0, len(user_ids), batch_size):
    batch_ids = user_ids[i:i+batch_size]
    data = models.execute_kw(
        db, uid, password,
        'res.users', 'read',
        [batch_ids],
        {'fields': fields_to_read}
    )
    records.extend(data)

# ===============================
# 🧾 Convertir a DataFrame
# ===============================
df = pd.DataFrame(records)

# Limpiar columnas booleanas
for c in group_fields:
    df[c] = df[c].fillna(False).astype(bool)

print(f"✅ DataFrame listo con {len(df)} usuarios y {len(fields_to_read)} columnas.")


Total de campos a leer: 134
✅ DataFrame listo con 397 usuarios y 134 columnas.


In [2]:
df.head()

,in_group_127,sel_groups_12,in_group_58,in_group_59,in_group_53,in_group_54,in_group_55,in_group_56,in_group_57,in_group_149,in_group_148,sel_groups_63_64,in_group_146,sel_groups_84,in_group_239,in_group_238,in_group_237,in_group_236,in_group_235,in_group_234,in_group_233,in_group_232,in_group_231,in_group_230,sel_groups_86_87,in_group_69,in_group_65,in_group_66,in_group_61,in_group_60,in_group_62,name,sel_groups_105_106,in_group_15,in_group_130,in_group_131,in_group_132,in_group_133,in_group_136,in_group_137,in_group_208,in_group_209,sel_groups_141,in_group_72,in_group_70,in_group_71,in_group_240,in_group_75,in_group_129,in_group_128,in_group_8,in_group_6,in_group_122,in_group_5,in_group_2,in_group_126,in_group_125,in_group_1,in_group_215,in_group_214,in_group_216,in_group_211,in_group_210,in_group_213,in_group_212,in_group_218,active,login,email,in_group_16,in_group_17,in_group_11,in_group_19,in_group_101,in_group_100,in_group_102,in_group_108,sel_groups_82_83,in_group_88,in_group_85,in_group_29,in_group_28,in_group_21,sel_groups_9_18_10,in_group_23,in_group_22,in_group_25,in_group_24,in_group_27,in_group_26,sel_groups_111_112,sel_groups_13_14,in_group_242,in_group_243,sel_groups_217,in_group_241,in_group_98,in_group_99,in_group_94,in_group_97,in_group_92,in_group_93,in_group_38,in_group_39,in_group_36,in_group_37,in_group_34,in_group_35,in_group_32,in_group_33,in_group_30,in_group_31,id,sel_groups_227,sel_groups_51_52,in_group_20,sel_groups_40_41,in_group_48,in_group_43,in_group_42,in_group_47,in_group_46,in_group_45,in_group_44,sel_groups_3_4,in_group_151,in_group_228,in_group_229,in_group_7,in_group_224,in_group_220,in_group_221,in_group_222,in_group_223
0,True,True,False,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False,True,True,True,True,False,True,True,True,Administrador Administrador,True,True,True,True,True,True,True,True,False,True,True,True,True,True,False,True,True,True,True,False,True,True,False,True,True,False,True,True,True,False,False,False,False,True,True,admin,odoo@donsson.com,False,True,True,False,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,False,True,True,False,False,True,False,False,False,False,True,False,True,False,True,False,True,False,False,False,True,True,False,1,False,True,True,True,False,False,False,True,True,False,True,True,True,True,True,True,True,False,True,False,True
1,False,False,False,True,True,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,True,True,LUIS IGNACIO ARANGO PINZON,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,ventasluis@donsson.com,ventasluis@donsson.com,False,True,True,False,False,False,False,False,False,False,False,True,True,True,True,True,True,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,260,False,False,True,True,False,False,False,True,True,False,True,False,False,False,False,True,False,False,False,False,False
2,False,False,False,True,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,True,True,ANA MARIA BETANCOURT,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False,True,True,False,False,False,False,False,False,False,False,False,True,ambetancourt@donsson.com,anama.betanc@gmail.com,False,True,True,False,False,False,False,False,False,True,True,True,True,True,True,True,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,False,False,False,True,True,False,266,False,False,True,False,False,False,False,Tru

In [30]:
import xmlrpc.client
import pandas as pd

# =====================================
# 🔐 Conexión a Odoo
# =====================================
url = "https://donsson.com"
db = "Donsson_produccion"
username = "juan.cano@donsson.com"
password = "1000285668"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# =====================================
# 2️⃣ Leer empleados con su cargo
# =====================================
employees = models.execute_kw(
    db, uid, password,
    'hr.employee', 'search_read',
    [[['active', 'in', [True]]]],
    {'fields': ['user_id', 'job_id','name']}
)
df_emp = pd.DataFrame(employees)

# Desanidar campos many2one
df_emp['user_id'] = df_emp['user_id'].apply(lambda x: x[0] if isinstance(x, list) else None)
df_emp['cargo'] = df_emp['job_id'].apply(lambda x: x[1] if isinstance(x, list) else None)
df_emp = df_emp[['name' ,'cargo']]

In [29]:
df_emp[df_emp["name"].str.contains("HAROLD")]

,name,cargo,name_norm
42,HAROLD STEVEN JIMENEZ RINCON,INGENIERO DE VENTAS,harold steven jimenez rincon


In [4]:
cols_grupos = [c for c in df.columns if c.startswith("in_group_") or c.startswith("sel_groups_")]
df_grupos = df[cols_grupos].astype(float).fillna(0)  # Convertir

In [5]:
df_grupos.head()

,in_group_127,sel_groups_12,in_group_58,in_group_59,in_group_53,in_group_54,in_group_55,in_group_56,in_group_57,in_group_149,in_group_148,sel_groups_63_64,in_group_146,sel_groups_84,in_group_239,in_group_238,in_group_237,in_group_236,in_group_235,in_group_234,in_group_233,in_group_232,in_group_231,in_group_230,sel_groups_86_87,in_group_69,in_group_65,in_group_66,in_group_61,in_group_60,in_group_62,sel_groups_105_106,in_group_15,in_group_130,in_group_131,in_group_132,in_group_133,in_group_136,in_group_137,in_group_208,in_group_209,sel_groups_141,in_group_72,in_group_70,in_group_71,in_group_240,in_group_75,in_group_129,in_group_128,in_group_8,in_group_6,in_group_122,in_group_5,in_group_2,in_group_126,in_group_125,in_group_1,in_group_215,in_group_214,in_group_216,in_group_211,in_group_210,in_group_213,in_group_212,in_group_218,in_group_16,in_group_17,in_group_11,in_group_19,in_group_101,in_group_100,in_group_102,in_group_108,sel_groups_82_83,in_group_88,in_group_85,in_group_29,in_group_28,in_group_21,sel_groups_9_18_10,in_group_23,in_group_22,in_group_25,in_group_24,in_group_27,in_group_26,sel_groups_111_112,sel_groups_13_14,in_group_242,in_group_243,sel_groups_217,in_group_241,in_group_98,in_group_99,in_group_94,in_group_97,in_group_92,in_group_93,in_group_38,in_group_39,in_group_36,in_group_37,in_group_34,in_group_35,in_group_32,in_group_33,in_group_30,in_group_31,sel_groups_227,sel_groups_51_52,in_group_20,sel_groups_40_41,in_group_48,in_group_43,in_group_42,in_group_47,in_group_46,in_group_45,in_group_44,sel_groups_3_4,in_group_151,in_group_228,in_group_229,in_group_7,in_group_224,in_group_220,in_group_221,in_group_222,in_group_223
0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0
1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1

In [6]:
# Número de grupos por usuario
df["n_grupos"] = df_grupos.sum(axis=1)

# Ver top 10 usuarios con más accesos
usuarios_acceso_total = df[["name", "login", "n_grupos"]].sort_values("n_grupos", ascending=False)
users_donsson = usuarios_acceso_total[usuarios_acceso_total["n_grupos"]>25]
users_customer = usuarios_acceso_total[usuarios_acceso_total["n_grupos"]<=25]
users_donsson.head(20)

/tmp/ipykernel_32912/146320959.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["n_grupos"] = df_grupos.sum(axis=1)


,name,login,n_grupos
0,Administrador Administrador,admin,85.0
4,JUAN CARLOS BETANCOURT GUTIERREZ,jbetancourt@donsson.com,77.0
20,ALBA LUCIA CASASBUENAS LOPEZ,financiero@donsson.com,67.0
49,CAROLINA RODRIGUEZ,contabilidad@donsson.com,63.0
3,GERMAN RICARDO BETANCOURT GUTIERREZ,gbetancourt@donsson.com,63.0
33,Simon Betancourt Del pino,sbetancourt@donsson.com,63.0
283,JULIANA RODRIGUEZ FRANCO,juliana.rodriguez@donsson.com,61.0
36,ANDREA ROBLEDO GIL,gestionhumana@donsson.com,59.0
15,JUAN IGNACIO NIETO NEIVA,logistica@donsson.com,58.0
18,LEIDY ANDREA SOTELO CRISTANCHO,asistentecontable@donsson.com,57.0


In [24]:
import pandas as pd
import unicodedata
from rapidfuzz import process, fuzz

# --- Función de normalización ---
def normalizar_texto(texto):
    if pd.isna(texto):
        return ''
    texto = texto.lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', errors='ignore').decode('utf-8')
    return ' '.join(texto.split())

# Normalizar nombres
users_donsson['name_norm'] = users_donsson['name'].apply(normalizar_texto)
df_emp['name_norm'] = df_emp['name'].apply(normalizar_texto)



matches = []
for name in users_donsson['name_norm']:
    match = process.extractOne(name, df_emp['name_norm'], scorer=fuzz.partial_ratio)
    if match and match[1] > 20:  # solo si se parece al menos en 80%
        matches.append(match[0])
    else:
        matches.append(None)

users_donsson['match_name_emp'] = matches
df_merge = users_donsson.merge(df_emp, left_on='match_name_emp', right_on='name_norm', how='left')


/tmp/ipykernel_32912/1532636539.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_donsson['name_norm'] = users_donsson['name'].apply(normalizar_texto)
/tmp/ipykernel_32912/1532636539.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_donsson['match_name_emp'] = matches


In [25]:
df_merge[df_merge["match_name_emp"].isna()]

,name_x,login,n_grupos,name_norm_x,match_name_emp,name_y,cargo,name_norm_y


In [28]:
import pandas as pd

# Agrupar por cargo
df_resumen = (
    df_merge.groupby("cargo")
    .agg({
        "name_x": lambda x: ", ".join(x),   # lista de nombres unidos por coma
        "n_grupos": "mean"                  # promedio o también puede ser list(x)
    })
    .reset_index()
)

# Si prefieres ver la lista de nombres más limpia (una por línea)
df_resumen["name_x"] = df_resumen["name_x"].str.replace(",", "\n-")

# Mostrar resultado
df_resumen


,cargo,name_x,n_grupos
0,ANALISTA CONTABLE,JULIANA RODRIGUEZ FRANCO,61.000000
1,ANALISTA DE COMPRAS,DANIEL FELIPE JARAMILLO BUITRAGO,53.000000
2,ANALISTA DE CRÉDITO Y CARTERA,VIVIANA PAYA,43.000000
3,ANALISTA DE DATOS,JUAN PABLO CANO CHAPARRO,52.000000
4,APRENDIZ SENA,AURELYSMAR REA SILVA,36.000000
5,ASISTENTE GERENCIA ADMINISTRATIVA,LEIDY ANDREA SOTELO CRISTANCHO,57.000000
6,ASISTENTE GERENCIA COMERCIAL Y MERCADEO,ALEXANDRA URRIBARRI,51.000000
7,AUXILIAR BODEGA MOSTRADOR,ANDRES CORRALES\n- OMAR CARRILLO,31.500000
8,AUXILIAR DE FACTURACIÓN Y DESPACHOS,YANID AIMEETH RODRIGUEZ JOTA,48.000000
9,CONDUCTOR,CARLOS ROBERTO GUTIERREZ,29.000000
